## Etapa atual
- **Validação da ideia**: criação de MVP manual para comparação de preços.  
- **Criação de base inicial**: coletar dados de algumas notas fiscais ou produtos.  
- **Objetivo desta etapa**: ter **uma base inicial de preços** para análises e comparativos.

##  Estrutura para etapa atual
1. Criar **script para raspar dados das notas fiscais**:  
   - Extrair: produto, preço, mercado, data.  
2. Alimentar **planilha ou banco de dados simples** (CSV / Google Sheets / SQLite).  
3. Fazer **comparativo de preços iniciais** usando esses dados.  
4. Validar se o MVP manual fornece resultados confiáveis antes de automatizar scraping de sites ou QR Codes.


In [6]:
import requests
from bs4 import BeautifulSoup

In [21]:
response = requests.get("https://www.nfce.fazenda.sp.gov.br/NFCeConsultaPublica/Paginas/ConsultaQRCode.aspx?p=35250944914620000101651020000029821102030260|2|1|1|7CEABA67901306C7669AF902F102EC944F682413")

html = response.text

soup = BeautifulSoup(html, 'html.parser')

In [22]:
paragrafos = soup.find_all('p')

In [28]:
div_id = soup.find('div', id='u20')

In [29]:
div_id

<div class="txtTopo" id="u20">MERCADO JARDIM ZAIRA LTDA</div>

In [43]:
nome_produto = soup.find('span', class_='txtTit')

print(nome_produto)

<span class="txtTit">REQUEIJAO TIROLEZ  CREMOSO 200GR</span>


In [42]:
qtd_comprada = soup.find('span', class_='Rqtd')

print(qtd_comprada)

<span class="Rqtd">
<strong>Qtde.:</strong>1</span>


In [45]:
unidade= soup.find('span', class_='RUN')

print(unidade)

<span class="RUN">
<strong>UN: </strong>UN0001</span>
